In [ ]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
# from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
import pickle

In [1]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

Host Service via Anvil

In [ ]:
class MomStrat():
    def __init__(self,th=.00025,regime='trend'):
        #regime can be 'trend' or 'mr' (mean-reversal)
        self.logL=[]
        self.th=th
        self.model_type='rule_based'
        self.data_cols=['Close','datetime']
        self.regime=regime
    def set_alt_data(self,alt_data_func,remote=False):
        if remote: self.gdata=anvil.server.call(alt_data_func)['gdata']
        else: self.gdata=alt_data_func()['gdata']
    def check_entry_batch(self,dfD):
        return always_buy(dfD)
        decisionsD={t:0 for t in dfD}
        stopD={t:5 for t in dfD}
        targetD={t:5 for t in dfD}
        dataD={}
        log_entry={}
        for t in dfD.keys():
            data=dfD[t]
            if data.shape[0]>2:
                r=np.log(data['Close']/data['Close'].shift(1)).dropna()
                m=r.rolling(30).mean().values
                if m[-1]>self.th: 
                    if self.regime=='trend':decisionsD[t]=1
                    elif self.regime=='mr':decisionsD[t]=-1
                elif m[-1]<-self.th: 
                    if self.regime=='trend':decisionsD[t]=-1
                    elif self.regime=='mr':decisionsD[t]=1
                else: decisionsD[t]=0
        return decisionsD,stopD,targetD
    def check_exit_batch(self,dfD,posf):
        def exit_fn(row):
            return self.exit_predicate(row,dfD[row.ticker])
        posf['to_exit']=posf.apply(exit_fn,axis=1).values
        return posf
    def exit_predicate(self,row,df):
        return False
        data=df
        r=np.log(data['Close']/data['Close'].shift(1)).dropna()
        m=r.rolling(20).mean().values
        if row['quant']>0 and m[-1]<self.th/3: return True
        elif row['quant']<0 and m[-1]>-self.th/3: return True
        else: return False
    def Check(strat,dfD):
        return strat.check_entry_batch(dfD)
    def Exit(strat,dfD,posf):
        return strat.check_exit_batch(dfD,posf)